<a href="https://github.com/xuehangcang/DeepLearning/blob/main/docs/PyTorch/%E5%9F%BA%E7%A1%80%E7%9F%A5%E8%AF%86/7.优化模型参数.ipynb" download=""><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="110" height="20" role="img" aria-label="jupyter: notebook"><title>jupyter: notebook</title><linearGradient id="s" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1"/><stop offset="1" stop-opacity=".1"/></linearGradient><clipPath id="r"><rect width="110" height="20" rx="3" fill="#fff"/></clipPath><g clip-path="url(#r)"><rect width="49" height="20" fill="#555"/><rect x="49" width="61" height="20" fill="#fe7d37"/><rect width="110" height="20" fill="url(#s)"/></g><g fill="#fff" text-anchor="middle" font-family="Verdana,Geneva,DejaVu Sans,sans-serif" text-rendering="geometricPrecision" font-size="110"><text aria-hidden="true" x="255" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="390">jupyter</text><text x="255" y="140" transform="scale(.1)" fill="#fff" textLength="390">jupyter</text><text aria-hidden="true" x="785" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="510">notebook</text><text x="785" y="140" transform="scale(.1)" fill="#fff" textLength="510">notebook</text></g></svg></a>

## 7.1 优化模型参数


现在，我们已经有了一个模型和数据，是时候通过优化其参数在数据上训练、验证和测试我们的模型了。训练模型是一个迭代的过程；在每次迭代中，模型猜测输出，计算其猜测的误差（*损失*），收集关于其参数的误差的导数，并使用梯度下降法**优化**这些参数。

## 7.2 先决条件代码

我们加载了前面部分关于`数据集和数据加载器`和`构建模型 `的代码。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 7.3 超参数

超参数是可调参数，可以让你控制模型优化过程。
不同的超参数值可以影响模型训练和收敛速度

我们定义以下超参数进行训练：

- **Epochs 数量** - 迭代数据集的次数
- **Batch Size** - 在更新参数之前，通过网络传播的数据样本数量
- **Learning Rate** - 每个批次/epoch 更新模型参数的程度。较小的值会产生较慢的学习速度，而较大的值可能会导致训练过程中出现不可预测的行为。

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 7.4 优化循环

一旦我们设置了超参数，我们就可以使用优化循环来训练和优化我们的模型。优化循环的每个迭代被称为一个 **epoch**。

每个 epoch 包含两个主要部分：

 - **训练循环** - 迭代训练数据集并尝试收敛到最优参数。
 - **验证/测试循环** - 迭代测试数据集以检查模型性能是否提高。

## 7.5 损失函数

当我们提供一些训练数据时，我们的未经训练的网络可能无法给出正确的答案。**损失函数** 用于衡量获取的结果与目标值之间的差异程度，我们希望在训练过程中最小化损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。


常见的损失函数包括用于回归任务的 `nn.MSELoss` 均方误差，以及用于分类的 `nn.NLLLoss` 负对数似然 和 `nn.CrossEntropyLoss`交叉熵损失，其中 `nn.CrossEntropyLoss` 结合了 `nn.LogSoftmax` 和 `nn.NLLLoss`。

我们将模型的输出 logits 传递给 `nn.CrossEntropyLoss`，该函数将标准化 logits 并计算预测误差。

In [3]:
loss_fn = nn.CrossEntropyLoss()

## 7.6 优化器

优化是通过调整模型参数来减少每个训练步骤中的模型误差的过程。**优化算法**定义了此过程的执行方式（在此示例中，我们使用了随机梯度下降法）。

所有优化逻辑都封装在 `optimizer` 对象中。在这里，我们使用了SGD优化器；此外，在PyTorch中还有许多不同的优化器可供选择，如 ADAM 和 RMSProp ，它们适用于不同类型的模型和数据。

我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环内部，优化分为三个步骤：

1. 调用`optimizer.zero_grad()`来重置模型参数的梯度。梯度默认会相加，为了防止重复计算，在每次迭代时我们显式地将它们清零。
2. 通过调用`loss.backward()`反向传播预测损失。PyTorch会保存损失对每个参数的梯度。
3. 一旦我们获得了梯度，我们调用`optimizer.step()`根据反向传播收集的梯度来调整参数。

完整的实现：

我们定义`train_loop`循环我们的优化代码，`test_loop`评估模型对测试数据的表现。

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化了损失函数和优化器，并将其传递给 `train_loop` 和 `test_loop`。如果需要追踪模型的性能改进，可以随意增加训练轮数。

In [6]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.319417  [   64/60000]
loss: 2.294992  [ 6464/60000]
loss: 2.279560  [12864/60000]
loss: 2.263008  [19264/60000]
loss: 2.243725  [25664/60000]
loss: 2.220730  [32064/60000]
loss: 2.218863  [38464/60000]
loss: 2.195359  [44864/60000]
loss: 2.190009  [51264/60000]
loss: 2.145596  [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 2.148571 

Epoch 2
-------------------------------
loss: 2.161798  [   64/60000]
loss: 2.146987  [ 6464/60000]
loss: 2.086889  [12864/60000]
loss: 2.106176  [19264/60000]
loss: 2.047474  [25664/60000]
loss: 1.985205  [32064/60000]
loss: 2.013071  [38464/60000]
loss: 1.934789  [44864/60000]
loss: 1.944472  [51264/60000]
loss: 1.861014  [57664/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.868127 

Epoch 3
-------------------------------
loss: 1.899601  [   64/60000]
loss: 1.863389  [ 6464/60000]
loss: 1.746067  [12864/60000]
loss: 1.797285  [19264/60000]
loss: 1.678444  [25664/60000]
loss: 1.627126  [32064/600